# 0. Google Drive Mount

In [1]:
from google.colab import drive
!drive.mount('/content/drive/suyun_speech_corpus')

/bin/bash: -c: line 1: syntax error near unexpected token `'/content/drive/suyun_speech_corpus''
/bin/bash: -c: line 1: `drive.mount('/content/drive/suyun_speech_corpus')'


# 1. Settings

In [2]:
# Install selenium
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [3]:
# Confirm Selenium
import selenium
print(selenium.__version__)

4.31.1


In [4]:
# Install chromedriver : Only once

#!apt-get update
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/suyun_speech_corpus'

In [5]:
# Install autoinstaller : to match the version
!pip install chromedriver_autoinstaller

In [6]:
# selenium으로 키를 조작하기 위한 import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ETC
import time # to wait not to seem as "bot"
import sys
import chromedriver_autoinstaller
import time
import csv

In [7]:
chrome_path = "/content/drive/MyDrive/suyun_speech_corpus/chromedriver"

In [8]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

# 3. Crawling - 역대 대통령
https://www.pa.go.kr/research/contents/speech/index.jsp

In [9]:
# search speech data of specific president
def search(president):
    name_d = {"전체":0, "이승만":1, "윤보선":2, "박정희":3, "최규하":4, "전두환":5,
              "노태우":6, "김영삼":7, "김대중":8, "노무현":9, "이명박":10, "박근혜":11,
              "문재인":12}
    presid_num = name_d[f"{president}"]

    checkbox = driver.find_element(By.XPATH, f'//*[@id="prstNm{presid_num}"]')
    search = driver.find_element(By.XPATH, '//*[@id="frm"]/fieldset[1]/div[2]/div[5]/button[2]')
    checkbox.click()
    time.sleep(1)
    search.click()
    time.sleep(1)
    doc_button = driver.find_element(By.XPATH, '//*[@id="m-items2"]/li[2]/a')
    doc_button.click()
    time.sleep(2)

In [10]:
# Helper function to click page button
def click_section_button():
    global driver
    try:
        page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
            EC.element_to_be_clickable((By.XPATH, f'//*[@id="frm"]/nav[2]/ul/li[13]/a')))
        driver.execute_script("arguments[0].scrollIntoView();", page_button) # scroll
        time.sleep(2)
        driver.execute_script("arguments[0].click();", page_button)  # Click with JavaScript
        time.sleep(2)
    except TimeoutException:
        print(f"Section button not found")
        raise  # if TimeoutException : throw Exception & quit

In [11]:
# Helper function to click page button
def click_page_button(page_num):
    rest = page_num % 10
    num = rest if rest != 0 else 10
    global driver

    try:
        if num == 1:
            page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="frm"]/nav[2]/ul/li[{num+2}]/strong')))
        else:
            page_button = WebDriverWait(driver, 10).until( # Waiting 10sec maximum
                EC.element_to_be_clickable((By.XPATH, f'//*[@id="frm"]/nav[2]/ul/li[{num+2}]/a')))

        driver.execute_script("arguments[0].scrollIntoView();", page_button) # scroll
        time.sleep(1)
        driver.execute_script("arguments[0].click();", page_button)  # Click with JavaScript
    except TimeoutException:
        print(f"Page button {num} not found")
        raise  # if TimeoutException : throw Exception & quit

### 문재인

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "문재인"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 70):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 제19대 대통령 취임식
2. 비정규직 근로자와의 대화
3. 미세먼지 바로알기 교실 현장 방문
4. 국방부 순시
5. 합동참모본부 순시
6. 제37주년 5·18민주화운동 기념식
7. 노무현 대통령 서거 8주기 추도식
8. 수석·보좌관회의
9. 수석·보좌관회의
10. 제22회 바다의 날 기념식
11. 수석·보좌관회의
12. OECD 국제교통포럼(ITF) 사무총장 당선 축전
13. 치매국가책임제 SNS 메시지
14. 치매국가책임제 관련 현장 방문
15. 제62회 현충일 추념식
16. 용산소방서 방문 후 SNS 메시지
17. 현장 소방대원들과의 대화
18. 국가안전보장회의(NSC)
19. 제30주년 6·10민주항쟁 기념식
20. 2017년도 추가경정예산안 관련 국회 시정 연설
21. 한미연합사령부 순시
22. 제19회 미주한인체전 축전
23. 제1회 전국 시·도지사 간담회
24. 6·15남북공동선언 17주년 기념식
25. 수석·보좌관회의
26. 아시아인프라투자은행(AIIB) 연차총회 개회식
27. 원자력발전소 고리1호기 영구 정지 선포식
28. 일자리위원회 위원 위촉장 수여식 및 제1차 회의
29. 6·25 한국전쟁 제67주년 국군 및 유엔군 참전유공자 위로연
30. 2017 무주 세계태권도선수권대회 개막식
31. 제28회 국무회의
32. (미국 방문) 장진호 전투 기념비 헌화
33. (미국 방문) 한미 비즈니스 서밋
34. 2017 글로벌 바이오 콘퍼런스 영상 축사
35. 2017 사회적경제 주간 기념 사회적경제 박람회 개막식 영상 축사
36. 한국신문협회 창립 60주년 영상 축사
37. (미국 방문) CSIS 전문가 초청 만찬
38. (미국 방문) 한미 정상 공동 언론 발표
39. (미국 방문) 워싱턴 동포 간담회
40. 방미 귀국 인사
41. 제50회 산업안전보건의 날 기념식 영상 축사
42. (독일 방문) 독일 동포 간담회
43. (독일 방문) 독일 의료 지원단 접견
44. (독일 방문) 한독 정상 공동 언론 발표
45. (독일 방문) 쾨르버재단 초청 연

In [ ]:
# 70 페이지
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

idx = 1392
president = "문재인"
#str(input("President's name you want : "))
search(president)

button = driver.find_element(By.XPATH, '//*[@id="frm"]/nav[2]/ul/li[14]/a')
button.click()
time.sleep(2)

for title_num in range(1, 13):
    clik = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
    driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
    time.sleep(1)
    driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

    o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
    o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
    o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
    o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

    title = o_title.text
    ttype = o_type.text
    date = o_date.text
    speech = o_speech.text
    speaker = president

    d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
    speech_data.append(d)
    print(f"{idx}. {title}")
    idx += 1
driver.quit()

1392. 국군간호사관학교 제62기 졸업식 및 임관식 축전


TimeoutException: Message: 
Stacktrace:
#0 0x58187b8a46aa <unknown>
#1 0x58187b575600 <unknown>
#2 0x58187b5c4bcb <unknown>
#3 0x58187b5c4eb1 <unknown>
#4 0x58187b608b24 <unknown>
#5 0x58187b5e78cd <unknown>
#6 0x58187b60604a <unknown>
#7 0x58187b5e7643 <unknown>
#8 0x58187b5b7d31 <unknown>
#9 0x58187b5b879e <unknown>
#10 0x58187b86c23b <unknown>
#11 0x58187b8701d2 <unknown>
#12 0x58187b8595f5 <unknown>
#13 0x58187b870d62 <unknown>
#14 0x58187b83e23f <unknown>
#15 0x58187b893e48 <unknown>
#16 0x58187b894020 <unknown>
#17 0x58187b8a347c <unknown>
#18 0x7da77625fac3 <unknown>


In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/since_president_MoonJaeIn.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 1392_문재인

### 박근혜

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "박근혜"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 25):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 제18대 대통령 취임식
2. 제94주년 3.1절 기념식
3. 대국민 담화
4. 국군 장교 합동 임관식
5. 제29기 경찰대학 졸업 및 임용식
6. 천안함 용사 3주기 추모식
7. 지역일자리정책한마당 축하 영상 메시지
8. 충남도청 신청사 개청식
9. 한반도 평화와 국민행복을 위한 대법회
10. 바르게살기운동 전국여성지도자대회 축하 영상 메시지
11. 제11회 할리웃보울 음악대축제 축하 영상 메시지
12. 제33회 장애인의 날 축하 영상 메시지
13. 청소년.가정의 달 기념 축하 영상 메시지
14. 한국여성정책연구원 개원 30주년 축하 영상 메시지
15. 과학기술인.정보통신인 한마음대회
16. 제50회 법의 날 기념식
17. 어린이날 축하 영상 메시지
18. 중앙선거관리위원회 창설 제50주년 및 제2회 유권자의 날 축하 영상 메시지
19. 미 상.하원 합동회의 연설
20. 미국상공회의소 주최 오찬
21. 제48회 발명의 날 기념식
22. 제33주년 5.18민주화운동 기념식
23. 디지털 케이블TV 쇼 축하 영상 메시지
24. 바다의 날 축하 영상 메시지
25. 한국청소년적십자 창립 제60주년 및 Global Peace 전국합동입단선서식 축하 영상 메시지
26. 한국형 기동헬기 '수리온' 전력화 기념 행사
27. 대구국가산업단지 기공식
28. 제18회 환경의 날 기념식
29. 제58주년 현충일 추념식
30. 세계한인회장대회 개회식
31. 국군 및 유엔군 6.25전쟁 참전유공자 위로연
32. 북경지역 재중 한국인 간담회
33. 한중 비즈니스포럼 연설
34. 칭화대학교 연설
35. 시안 지역 한국인 오찬 간담회
36. 제18회 여성주간 기념식
37. '내 일' 행복자활박람회
38. 제2회 정보보호의 날 축하 영상 메시지
39. 제17회 여성경제인의 날 기념식
40. 제16차 월남 참전 고엽제 전우 충혼 위령제 영상 메시지
41. 유엔군 참전, 정전 제60주년 유엔 참전용사 추모식
42. 세계한인입양인협회 Gathering 2013 축하 영상 메시지
43. 제14

In [ ]:
# 25 페이지
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

president = "박근혜"
search(president)

button = driver.find_element(By.XPATH, '//*[@id="frm"]/nav[2]/ul/li[14]/a')
button.click()
time.sleep(2)

for title_num in range(1, 3):
    clik = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
    driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
    time.sleep(1)
    driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

    o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
    o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
    o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
    o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

    title = o_title.text
    ttype = o_type.text
    date = o_date.text
    speech = o_speech.text
    speaker = president
    print(f"{idx}. {title}")
    driver.back()
driver.quit()

482. 제97회 전국체육대회 개회식
482. 재일민단 창단 70주년 기념식


In [ ]:
# 25 페이지
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

idx = 482
president = "박근혜"
search(president)

button = driver.find_element(By.XPATH, '//*[@id="frm"]/nav[2]/ul/li[14]/a')
button.click()
time.sleep(2)

for title_num in range(2, 14):
    clik = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
    driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
    time.sleep(1)
    driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

    o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
    o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
    o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
    o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

    title = o_title.text
    ttype = o_type.text
    date = o_date.text
    speech = o_speech.text
    speaker = president

    d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
    speech_data.append(d)
    print(f"{idx}. {title}")
    idx += 1
    driver.back()
driver.quit()

482. 재일민단 창단 70주년 기념식
483. 제17회 세계지식포럼 개막식
484. 제34회 대통령기 이북도민 체육대회
485. 제50회 전국 상이군경체육대회 개막식
486. 지구촌 새마을지도자대회
487. 도레이첨단소재 구미4공장 기공식
488. 대한민국 행복교육 박람회
489. 경찰의 날
490. 2017 예산안 시정연설
491. 글로벌 인재포럼 2016 개회식
492. 제4회 지방자치의 날 기념식 및 지방자치박람회
493. 제71주년 교정의 날 기념식


In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/since_president_ParkGeunHye.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 493개_박근혜

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/since_president_ParkGeunHye.csv


### 이명박

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "이명박"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 42):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 제17 대 대통령 취임사
2. 학군사관후보생 제46기 임관식 축사
3. 제89주년 3·1절 기념사
4. 2008 KR/FE 연습 격려편지
5. 한국노동조합총연맹 창립 제62주년 축사
6. 육군사관학교 제64기 졸업 및 임관식 축사
7. 육군3사관학교 제43기 졸업 및 임관식 축사
8. 제48주년 3·15의거 기념사
9. 해군사관학교 제62기 졸업 및 임관식 축사
10. 제35회 상공의 날 축사
11. 추비선 어린이에게 보내는 편지
12. 공군사관학교 제56기 졸업 및 임관식 축사
13. 2008 세계 물의 날 기념 청계천 걷기 대회 축사
14. 원담 대종사 조의메시지
15. 국민통합실천전국협의회 창립대회축사
16. 6·25 참전 유공자의 국가유공자 지정 경축행사 축사
17. 문화일보 지령 5,000호 발간 축하 메시지
18. 제40주년 향토 예비군의 날 축하 메시지
19. 제52회 신문의 날 기념사
20. 제36회 보건의 날 축사
21. 미·일 순방 관련 기자회견 모두 연설
22. [미국방문] 뉴욕동포 간담회 격려사
23. [미국방문] 코리아 소사이어티 주최 만찬사
24. 미·일 순방 출국 메시지
25. 영연방 UN 참전 용사 초청 환영만찬 격려사
26. [미국 방문] Global Korea: A Gateway to Asia
27. [미국방문] 뉴욕 한국투자설명회 연설
28. [미국방문] 워싱턴 동포 간담회 격려사
29. 제9회 서울국제금융포럼 개막식 축사
30. 2008 함평 세계 나비·곤충 엑스포 개막식 축사
31. 4·19혁명 제48주년 기념 국가조찬기도회 축사
32. [미국 방문] Towards a Renewed Partnership of Shared Values, Friendship, and Prosperity
33. [미국방문] 미 상공회의소 및 한·미 재계회의 공동 주최만찬사
34. 제28회 장애인의 날 기념사
35. [일본 방문]재일동포 간담회 격려사
36. [일본 방문]일본 경제단체 주최 오찬사
37. 나라기록관 개관식 축사
38. 원기 

In [ ]:
len(speech_data)

819

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/LeeMyungBak.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 819개_이명박

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/LeeMyungBak.csv


### 노무현

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "노무현"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 40):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 제16 대 대통령 취임 경축연 연설
2. 제16 대 대통령 취임사
3. 제16 대통령 취임 축하 외빈을 위한 만찬사
4. 2·28민주의거 43주년 기념메시지
5. 학군사관후보생(ROTC) 제41기 임관식 치사
6. 제84주년 3·1절 기념사
7. 참여정부 국정토론회 말씀
8. 육군사관학교 제59기 졸업 및 임관식 치사
9. 해군사관학교 제57기 졸업 및 임관식 치사
10. 3·15민주의거 43주년 기념 메시지
11. 공군사관학교 제51기 졸업 및 임관식 치사
12. 제30회 상공의 날 기념식 연설
13. 경찰대학 제19기 졸업 및 임용식 치사
14. 제주국제컨벤션센터 개관식 축하 메시지
15. 육군3사관학교 제38기 졸업 및 임관식 치사
16. 제238회 임시국회 국정연설
17. 향토예비군 창설 35주년 축하 메시지
18. 제1회 동북아경제포럼 축하 메시지
19. 한국형 구축함 문무대왕함 진수식 축사
20. 3자위원회 위원을 위한 만찬연설
21. 대한민국 임시정부 수립 84주년 기념식 연설
22. 한국전에서의 흑인용사들 기념행사 축하 메시지
23. 4·19혁명 43주년 기념 국가조찬기도회 축하 메시지
24. 부활절 축하 메시지
25. 제36회 과학의 날 기념식 연설
26. 새마을 운동 제창 33주년 축하 메시지
27. 제48회 정보통신의 날 기념식 연설
28. 이라크 파병부대 신고 및 환송식 치사
29. 불기2547년 부처님 오신 날 축하 메시지
30. 미주 한인 이민 100주년 기념축제축하 메시지
31. [미국 방문]출국인사
32. [미국 방문]코리아 소사이어티 초청 만찬연설
33. 스승의 날 사랑의 사이버 카네이션 행사 축하 메시지
34. [미국 방문]미국상공회의소, 한미재계회의 공동초청 오찬 연설
35. [미국 방문]우드로 윌슨센터, 국제전략문제연구소 초청 간담회 연설
36. [미국 방문]부시 미국 대통령 주최 만찬 답사
37. [미국 방문]귀국보고
38. 광주민주화운동 23주년 기념식 연설
39. 제38회 발명의 날 기념식 연설
40. 주한미국 전

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/RohMooHyun.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 780개_노무현

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/RohMooHyun.csv


### 김대중

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "김대중"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 43):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 대통령 취임 경축 만찬 연설(위기극복에 다함께 나섭시다)
2. 제15 대통령 취임사(국난극복과 재도약의 새시대를 엽시다)
3. 제79주년 3·1절 기념사(3·1운동의 대화합 정신으로 국난극복)
4. 98년도 학군사관후보생 임관식 연설(문무를 겸비한 새시대의 민주국군)
5. 김종필 총리서리 임명 관련 국민 여러분께 드리는 말씀(국정공백을 막기 위한 화합과 협력)
6. 신임 국무위원 임명장 수여 및 다과회 말씀(합심과 협력으로 국민에게 희망과 안정을)
7. 육군사관학교 제54기 졸업 및 임관식 연설(국민의 신뢰와 사랑을 받는 민주군대의 핵심)
8. 해군사관학교 제52기 졸업 및 임관식 연설(바다를 통한 세계화의 첨병)
9. 공군사관학교 제46기 졸업 및 임관식 연설(공군력은 현대전의 중심이자 핵심전략)
10. 경찰대학 제14기 졸업 및 임용식 연설(국민과 함께 하는 국민의 경찰)
11. 청와대 출입기자 초청 오찬 간담회 말씀(정치안정을 위한 야당의 협력 절실)
12. 제30회 국가조찬기도회 연설(구국의 결의와 실천을 다짐하며)
13. 제1회 무역·투자진흥대책회의시 말씀(지금의 난국을 재도약의 기회로)
14. 시·도 지사 오찬말씀(소신과 독자성 살린 지방행정 실시)
15. 제주 국제컨벤션센터 기공식 축하 메시지(세계평화와 발전을 이끄는 세계 평화의 섬)
16. 영국경제인협회(CBI)(한·영 공동이익을 위한 상호협력체제강화)
17. 런던 대학교 SOAS 초청강연회 연설(남북간 화해와 교류·협력 추진)
18. 제2차 ASEM 참석 귀국기자회견 서두말씀(한국에 고위급 투자사절단 파견)
19. 제42회 신문의 날 리셉션 말씀(비판없는 찬양보다는 우정있는 비판을)
20. 전국 검사장 오찬 말씀(국민의 신뢰와 존경받는 검찰상 정립)
21. 음베키 남아프리카공화국 부통령을 위한 오찬 연설(아프리카·아시아간 가교의 역할 기대)
22. 제30회 한일·일한 경제인회의 축하 메시지(한·일 경제협력의 새로운 전기 마련)
23. 4·19혁명 제38주년 국가조찬기도회 연설(숭고하게 피

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/KimDaeJung.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 822개_김대중

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data/KimDaeJung.csv


### 김영삼

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "김영삼"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 38):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 고촉통 싱가포르 총리 내외를 위한 만찬사(아·태 평화와 번영의 동반자)
2. 제14 대통령 취임사(우리 다 함께 신한국으로)
3. 첫 국무회의 말씀(국무위원은 개혁의 선봉장)
4. 제74주년 3·1절 기념사(한국병 치유로 신한국 창조)
5. 헬무트 콜 독일 총리를 위한 만찬사(공산주의는 지난 시대의 유물)
6. 경제장관회의 말씀(신경제5개년 계획 수립)
7. 육군사관학교 제49기 졸업 및 임관식 연설(문과 무는 한 뿌리의 한 나무)
8. 조선일보 창간 73주년 특별회견(윗물맑기 운동의 핵심은 돈 안드는 정치)
9. 대통령 취임 경축 특사 관련 국무회의 말씀(국민 대화합을 위한 사면·복권)
10. 공군사관학교 제41기 졸업 및 임관식 연설(각 군은 균형적 발전을 기해야)
11. 해군사관학교 제47기 졸업 및 임관식 연설(깨끗한 정부, 강력한 국군)
12. 국가기강 확립 보고회의 말씀(국가기강 확립은 내 자신의 개혁부터)
13. 신경제관련 특별담화문(신경제로 새로운 도약을)
14. 신경제100일 계획 보고회의 말씀(눈물과 땀으로 신경제건설)
15. 매일경제신문 창간 27주년 특별회견(작고 강력한 정부 실현)
16. 경찰대학 제9기 졸업 및 임용식 연설(경찰은 움직이는 작은 정부)
17. 간부 공무원에게 보내는 서한(국민에게 봉사하는 공무원)
18. 동아일보 창간73주년 특별회견(재산공개는 도덕정치를 위한 결단)
19. 경제장관회의 말씀(신경제100일 계획의 착실한 실천)
20. 제37회 신문의 날 기념 리셉션 연설(정치와 언론은 같은 배를 탄 동지)
21. 칼 빌트 스웨덴 총리를 위한 만찬사(한·스웨덴 관계발전을 위한 역사적 계기)
22. 민자당 제3차 상무위원회 총재 연설(세계 속에 우뚝 설 민족사의 새벽)
23. 중소기업 구조개선 보고대회 말씀(중소기업은 국가경제의 뿌리)
24. 재외 공관장을 위한 만찬 연설(돈 많은 사람이 부끄러운 시대)
25. 경제장관회의 말씀(신경제를 위한 규제완화에 역점)
26. 광주일보 창간 41주년 축하 메시지(언론이 자율과 

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/KimYoungSam.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 728개_김영삼

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data/KimYoungSam.csv


### 노태우

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "노태우"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 31):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 제13대 대통령 취임사
2. 제69주년 3·1절 기념사
3. 민주정의당 제13대 국회의원후보자 공천장 수여식 연설
4. 민주평화통일자문회의 오찬 연설
5. 공군사관학교 제36기 졸업 및 임관식 치사
6. 육군사관학교 제44기 졸업 및 임관식 치사
7. 민주정의당 지구당 창당대회 총재 치사
8. 재외공관장회의 참석자들을 위한 만찬 연설
9. 경찰대학 제4기 졸업 및 임관식 치사
10. 월간현대문학 통권400호 축하서한
11. 향토예비군 창설 제20주년 기념일에 즈음한 담화
12. 해군사관학교 제42기 졸업 및 임관식 치사
13. 광주·전남지역 각계인사 초청 다과회 연설
14. 민주평화통일자문회의 자문위원에게 보내는 친서
15. 취임 후 첫 기자회견
16. 서울지역 각계인사 초청 다과회 연설
17. 민주정의당 지구당위원장 접견시 연설
18. 소년조선일보 서면회견
19. 소년한국일보 서면회견
20. 예순 여섯번째 어린이 날을 맞아 어린이들에게 보내는 말씀
21. 청와대국무회의에서의 지시
22. 제13대 국회 개원식 연설
23. 미국 유 에스 에이 투데이지 회견
24. 한국일보 창간34돌 기념 특별회견
25. 6·29선언 1주년 기념민정신문특별회견
26. 6·29선언 1주년 기자간담회
27. 리관유 싱가폴 수상내외를 위한 만찬 만찬사
28. 민족자존과 통일번영을 위한 특별선언
29. 광양제철소 제2기 종합준공식 치사
30. 제43주년 광복절 경축사
31. 청와대 국무회의에서의 지시
32. 미국 뉴스위크지 회견
33. 미국 타임지 회견
34. 미국 NBC TV 회견
35. 일본 AERA지 서면회견
36. 서울올림픽 개막에 즈음한 특별담화
37. 서울올림픽 참관 외빈내외를 위한 만찬 연설
38. 프랑스르 몽드지 회견
39. 중앙일보 창간23주년 기념 특별회견
40. 독일 ZDF TV 회견
41. 서울올림픽 폐막에 즈음하여 국민께 드리는 감사말씀
42. 서울올림픽의 성공적 개최에 즈음하여 일본에 계시는 동포께 드리는 감사말씀
43. 서울올림픽의 성공적 개최에 즈음하여

In [ ]:
# 31 페이지
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

idx = 601
president = "노태우"
#str(input("President's name you want : "))
search(president)

button = driver.find_element(By.XPATH, '//*[@id="frm"]/nav[2]/ul/li[14]/a')
button.click()
time.sleep(2)

for title_num in range(1,2):
    clik = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
    driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
    time.sleep(1)
    driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

    o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
    o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
    o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
    o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

    title = o_title.text
    ttype = o_type.text
    date = o_date.text
    speech = o_speech.text
    speaker = president

    d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
    speech_data.append(d)
    print(f"{idx}. {title}")
    idx += 1
driver.quit()

601. 민주자유당(民主自由黨) 중앙당사(中央黨舍) 고별방문 연설


In [ ]:
len(speech_data)

601

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/RohTaeWoo.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 601개_노태

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data/RohTaeWoo.csv


### 전두환

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "전두환"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 32):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. [국보위상임위원장편]국가보위비상대책위원회 분과위원장 및 위원에 대한 훈시
2. [국보위상임위원장편]조찬기도회 인사말
3. [국보위상임위원장편]문화방송·경향신문 사장과의 특별회견
4. [국보위상임위원장편]전국대학총학장을 위한 오찬인사말
5. [국보위상임위원장편]전역사
6. 제11대 대통령당선에 즈음한 담화
7. 제11대 대통령 취임사
8. 육군제3사관학교 제17기 졸업식 및 임관식
9. 민방위대 창설 5주년기념 훈시
10. 후꾸다 다께오 전 일본수상을 위한 만찬 연설
11. 헌법개정안 공고에 즈음한 특별담화
12. 제32주년 국군의날 담화
13. 제4312주년 개천절 경축사
14. 1980년도 인구 및 주택센서스 실시에 즈음한 친서
15. 제61회 전국체육대회 치사
16. 국내기자회견
17. 제26회 백제문화제치사
18. 문화의날 기념식 치사
19. 제5공화국 헌법공포에 즈음한 담화
20. 제35주년 경찰의날 기념식 유시
21. 통일주체국민회의 종료행사 치사
22. 1980년도 추가경정예산안 제출에 즈음한 시정연설
23. 1981년도 예산안 제출에 즈음한 시정연설
24. 전국지방장관·시장·군수·구청장·경찰서장대회 유시
25. 북평항 제1단계 준공식 치사
26. 일본 NHK-TV와의 회견
27. 제61회 순국선열 합동추념식 메시지
28. 미국 월 스트리트 저널지와의 회견
29. 1980년도 농어업조사 실시에 즈음한 친서
30. 서울신문 특별회견
31. 정치활동재개에 즈음한 특별담화
32. 품질과 생산성향상을 위한 리셉션 격려
33. 근로자를 위한 리셉션 격려
34. 제17회 수출의날 기념식 치사
35. 대청다목적댐 준공식 치사
36. 국민교육헌장 선포 제12주년 기념식 치사
37. 전문경영인과 산학기술인을 위한 리셉션 격려
38. 전국새마을지도자대회 치사
39. 전국새마을지도자에게 보내는 친서
40. 해외건설업체 대표 리셉션 격려
41. 보건의료인을 위한 리셉션 격려
42. 주한 유엔군장병에게 보내는 성탄절 메시지
43. 공무원윤리헌장 선포식 및 우수공무원 

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/JeonDuhwuan.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 602개_전두환

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data/JeonDuhwuan.csv


### 최규하

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "최규하"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 4):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 국가비상시국에 관한 대통령권한대행 특별담화
2. 고 박정희대통령 대한민국 국장 조사
3. 시국에 관한 대통령권한대행 특별담화
4. 1980년도 예산안 제출에 즈음한 시정연설문
5. 통일주체국민회의 통일안보보고회 치사
6. 통일주체국민회의 대의원에게 보내는 친서
7. 대한적십자사 창립 제74주년 기념식 치사
8. 제16회 수출의 날 기념식 치사
9. 대통령후보 추천승낙에 즈음한 인사
10. 국민교육헌장 선포 제11주년 기념식 치사
11. 제10대 대통령 당선에 즈음한 인사
12. 제2대 통일주체국민회의 제3차회의 개회식 개회사
13. 대통령 긴급조치 제9호 해제에 즈음한 담화
14. 온산동제련소 준공식 치사
15. 통일주체국민회의 대의원들에게 보내는 친서
16. 통일주체국민회의 대의원들에게 보내는 친서
17. 제10대 대통령 취임사
18. 주한 유엔군장병에게 보내는 성탄절 메시지
19. 전국새마을지도자대회 치사
20. 1980년 신년사
21. 1980년 연두 기자회견
22. 1980년도 대간첩대책 중앙회의 유시
23. 여천석유화학공장 합동준공식 치사
24. 기강확립에 관한 특별지시
25. 국정자문회의 발족에 즈음한 인사
26. 긴급조치위반자 복권조치에 즈음한 담화
27. 제61주년 3·1절 기념사
28. 조선일보 창간 제60주년 축사
29. 한국정신문화연구원 부설 한국학대학원 개원식 및 1980년도 입학식 치사
30. 헌법개정심의안원회 개회식 인사
31. 전국지방장관·시장·군수·구청장 및 경찰서장회의 유시
32. 동아일보 창간 제60주년 축사
33. 육군사관학교 제36기 졸업식 유시
34. 공군사관학교 제28기 졸업식 유시
35. 제35회 식목일 담화문
36. 제24회 신문의 날 기념대회 치사
37. 한국형 전투구축함 울산함 진수식 치사
38. 해군사관학교 제34기 졸업식 유시
39. 최근의 내외정세에 관한 담화
40. 청평양수발전소 준공식 치사
41. 재일본 대한민국 거류민단 중앙본부 장총명 단장에게 보내는 친서
42. 새마을운동 10주년 담화문
43. 

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/ChoiGyuHa.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 58개_최규하

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data/ChoiGyuHa.csv


### 박정희

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "박정희"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 65):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 국회 개회식 치사
2. 제5대 대통령 취임식 대통령 취임사
3. 신년 메시지
4. 고급공무원에게 보내는 서신
5. 여당 국회의원에게 보내는 서신
6. 1964년 대통령 연두교서
7. 수협소식지에 실린 신년사
8. 자유의 날 10주년에 즈음한 메시지
9. 제2기 학도군사훈련생 임관식 유시
10. 서울대학교(大學校) 졸업식(卒業式) 치사(致辭)
11. 해군사관학교 제18기 졸업식 유시
12. 제45회 3·1절 경축사
13. 공군사관학교 졸업식 유시
14. 국가안전보장회의 훈시
15. 제6회 근로자의 날 메시지
16. 제6회 근로자의 날 치사
17. 식량증산 연찬대회 치사
18. 수운대신사 순도 100주년 기념 동상 제막식에 즈음한 치사
19. 한·일회담에 관한 특별담화문
20. 호남비료 나주공장 인수식 치사
21. 제19회 식목일 치사
22. 맥아더장군 서거에 대한 추도담화문
23. 제8회 신문의 날에 즈음한 메시지
24. 「맥아더」장군(將軍) 추도식(追悼式) 추도사(追悼辭)
25. 부산 문화방송국 개국 5주년 기념 치사
26. 쌍용양회공장 시무식 치사
27. 4·19의거 제4주년 기념식 기념사
28. 제2회 체육상 시상식 치사
29. 충무공 이순신장군 제419회 탄신일 기념사
30. 제2회 지방장관회단 유시
31. 제1회 법의 날 기념사
32. 환율제도 개혁에 관한 담화문
33. 제42회 어린이날 담화문
34. 울산정유공장 준공식 치사
35. 라디오 서울 개국 기념 치사
36. 제1회 상공인의 날 치사
37. 5·16혁명 제3주년 기념 메시지
38. 선열 석고상 건립 제막식 치사
39. 정치범 석방에 관한 담화문
40. 제3회 국제음악제치사
41. 군장비지원 낙동강유역 개간공사 기공식 치사
42. 제1회 중소기업자대상 치사
43. 제3회 향토문화 공로자 시상식 치사
44. 제3회 오월문예상 시상식 치사
45. 재일교포 올림픽후원회 결성에 즈음한 메시지
46. 비상계엄령 선포에 즈음한 담화문
47. 제9회 현충일 추념사
48. 제16회 근농절 치사
49. 1

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/ParkJeongHee.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 1270개_박정희

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data/ParkJeongHee.csv


### 윤보선

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "윤보선"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 2):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 제4대 윤보선 대통령 취임사
2. 윤보선 대통령 부산연설 "독재재기 경계"
3. 윤보선 대통령 대구연설 "도의심 앙양 강조"
ERROR OCCURED Message: 
Stacktrace:
#0 0x5b4ab058fc7a <unknown>
#1 0x5b4ab0272e2c <unknown>
#2 0x5b4ab02bf661 <unknown>
#3 0x5b4ab02bf751 <unknown>
#4 0x5b4ab0303f64 <unknown>
#5 0x5b4ab02e25ed <unknown>
#6 0x5b4ab0301303 <unknown>
#7 0x5b4ab02e2363 <unknown>
#8 0x5b4ab02b2247 <unknown>
#9 0x5b4ab02b2b9e <unknown>
#10 0x5b4ab055624b <unknown>
#11 0x5b4ab055a2f1 <unknown>
#12 0x5b4ab0541afe <unknown>
#13 0x5b4ab055ae52 <unknown>
#14 0x5b4ab052679f <unknown>
#15 0x5b4ab057f638 <unknown>
#16 0x5b4ab057f810 <unknown>
#17 0x5b4ab058edac <unknown>
#18 0x79fe08c30ac3 <unknown>



In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/YoonBoSun.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 602개_전두환

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data/YoonBoSun.csv


### 이승만

In [ ]:
# 10개 페이지들 --> 넘겨서 반복
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.pa.go.kr/research/contents/speech/index.jsp')
time.sleep(2)

speech_data = []
idx = 1
president = "이승만"
#str(input("President's name you want : "))
search(president)

#click_section_button()
for page_num in range(1, 51):
    click_page_button(page_num)
    for title_num in range(1, 21):
        try:
            clik = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="M_More"]/tr[{title_num}]/td[5]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", clik) # scroll
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clik)  # Click with JavaScript

            o_title = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[1]/td')
            o_type = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[3]/td[1]')
            o_date = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[2]/td[1]')
            o_speech = driver.find_element(By.XPATH, '//*[@id="content"]/div/table/tbody/tr[4]/td')

            title = o_title.text
            ttype = o_type.text
            date = o_date.text
            speech = o_speech.text
            speaker = president

            d = {"index": idx, "title": title, "type": ttype, "date": date, "speaker": president, "speech": speech}
            speech_data.append(d)
            print(f"{idx}. {title}")
            idx += 1

            driver.back()
        except Exception as e:
            print(f"ERROR OCCURED {str(e)}")
            break
    if page_num % 10 == 0 :
        click_section_button()
driver.quit()

1. 대통령 취임사(大統領就任辭)
2. 민족이 원하는 길을 따를 결심, 국무총리 인준 부결에 대하여
3. 미급점(未及點) 육성하라
4. 전민족에게 충고함
5. 대한민국 정부수립과 우리의 각오
6. 함께 뭉쳐서 자강전진(自疆前進) 외모(外侮)막자 (해방 3주년 기념사)
7. 정권 이양문제에 대하여
8. 북한정권(北韓政權)에 대하여
9. 미소(美蘇) 양군 철퇴문제에 대하여
10. 반민자(反民者) 처단(處斷)은 민의(民意), 법운영은 보복보다 개과천선(改過遷善)토록 하라
11. 미곡 수집에 대하여
12. 맥 원수(元帥) 한국(韓國) 보위(保衛)에 언급
13. 한글날을 맞이하여
14. 미국민(美國民)과 이해증진을 희망함
15. 한일통상에 대하여
16. 반도(叛徒)는 진퇴유곡(進退維谷) 후환(後患) 방어책에 전력하라
17. 적자(賊子)의 선동을 경계하고 충의로 치안유지에 협력하라
18. 불순배(不純輩)를 철저히 제거 반역사상방지법령 준비
19. 트루만 미국 대통령 당선에 대하여
20. 안전보장은 관민일치로
21. 공산분자의 반란 정부가 책임질 수 없다, 국회의 내각 개조요구는 유감
22. 양곡 매상 불여의(不如意) 시(時)는 특별 조처(措處)를 강구
23. 건설적인 비평과 파괴적인 비평
24. 국회와는 화해
25. 미곡 수집에 협력하라
26. 청년들은 궐기하여 반역분자 타도하라
27. 정부를 옹호하자
28. UN총회의 한국문제 토의 연기는 부당
29. 이 강토는 한족(韓族)이 주인이다
30. 민주와 공산은 상반된 이념, 중국적화는 용납할 수 없다
31. 통일에 협력요망
32. UN 정치위원회의 결의에 대하여
33. 신국면에 대하여
34. 석탄채굴에 힘쓰자
35. 이북동포와 인민군에게 보냄
36. 새해를 마지하는 마음
37. 평생에 기쁜 소식
38. 전국 청년단체 합동과 청년의 사명
39. 개각설은 무근(無根), 국민은 현혹치 마라
40. 쌀 암매장치(闇買藏置)는 부당, 호농지주의 협력이 긴요
41. 민생과 국방에 최중점
42. 화부(華府)에 대사관 설치함
43. 불법 

In [ ]:
# 저장할 경로 지정 (Google Drive 내 특정 폴더 경로)
folder_path = '/content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data'  # 'your_folder_name'을 실제 폴더 이름으로 변경
file_path = f'{folder_path}/LeeSeungMan.csv'

# CSV 파일로 저장
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["index", "title", "type", "date", "speaker", "speech"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in speech_data:
        writer.writerow(row)

print(f"SAVE COMPLETED: {file_path}") # 997개_이승만

SAVE COMPLETED: /content/drive/MyDrive/suyun_speech_corpus/corpus/Since_President/data/LeeSeungMan.csv
